In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
import yaml

mysql_info = yaml.safe_load(open('../mysql.yml'))

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}
data = []

teams = ['10001','1001','11001','12001','2002','3001','5002','6002','7002','9001','9002']

for team in teams:
    # 요청에 필요한 데이터 설정
    url = f'https://statiz.sporki.com/stats/?m=main&m2=batting&m3=default&so=&ob=&year=2024&sy=2019&ey=2024&te={team}&po=&lt=10100&reg=C5&pe=&ds=&de=&we=&hr=&ha=&ct=&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=1000&ph=&hs=&us=&na=&ls=1&sf1=WAROff&sk1=&sv1=&sf2=WAROff&sk2=&sv2='

    response = requests.post(url, headers=headers)  # 또는 requests.get() 사용

    res = requests.get(url, headers=headers)
    res.raise_for_status()

    html = response.text

    # 응답에서 데이터 추출
    soup = BeautifulSoup(html, "html.parser")

    # print(soup)

    rows = []

    trs = soup.find_all("tr")

    for tr in trs[2:]:
        tds = tr.find_all("td")
        player_id = tds[1].find("a").attrs['href'].split('/')[2].split('&')[1].split('=')[1]
        team_id = tds[2].find("img").attrs['src'].split('/')[5].split('.')[0]
        tds = [td.text for td in tds]
        tds.append(player_id)
        tds.append(team_id)
        rows.append(tds)

    for row in rows:
        temp = []
        for index in range(1, len(row)):
            if index == 2:
                season = row[index][:2]
                position = row[index][3:]
                temp.append(season)
                temp.append(position)
            else:
                temp.append(row[index])
        data.append(temp)

FileNotFoundError: [Errno 2] No such file or directory: './mysql.yml'

In [29]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

hitter_df = pd.DataFrame(data, columns=['이름', '연도', '포지션', 'WAR', 'oWAR', 'dWAR', 'G', 'PA', 'ePA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HP', 'IB', 'SO', 'GDP', 'SH', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'R/ePA', 'WRC+', 'DUMMY', 'P_CODE', 'T_CODE'])

hitter_df = hitter_df.dropna(axis=0)

hitter_df.tail()

,이름,연도,포지션,WAR,oWAR,dWAR,G,PA,ePA,AB,R,H,2B,3B,HR,TB,RBI,SB,CS,BB,HP,IB,SO,GDP,SH,SF,AVG,OBP,SLG,OPS,R/ePA,WRC+,DUMMY,P_CODE,T_CODE
1532,고종욱,21,LF,-0.65,0.36,-1.01,88,199,198,180,25,48,5,0,2,59,18,2,3,16,0,0,30,6,1,2,0.267,0.323,0.328,0.651,-0.025,82.9,-0.65,10659,9002
1533,전의산,24,1B,-0.74,-0.48,-0.27,16,35,35,32,1,3,0,0,1,6,2,0,0,3,0,0,9,1,0,0,0.094,0.171,0.188,0.359,-0.178,-35.9,-0.74,14699,9002
1534,정의윤,21,LF,-0.77,-0.04,-0.73,62,178,178,165,21,38,2,0,7,61,24,0,0,11,0,1,37,5,0,2,0.230,0.275,0.370,0.645,-0.051,62.0,-0.77,10655,9002
1535,이재원,23,C,-0.81,-0.74,-0.06,27,48,47,44,3,4,1,0,0,5,2,0,0,2,0,0,8,4,1,1,0.091,0.128,0.114,0.242,-0.198,-60.3,-0.81,10082,9002
1536,하재훈,24,RF,-1.42,0.04,-1.46,52,159,159,150,18,39,9,0,4,60,18,5,0,5,1,0,45,3,0,3,0.260,0.283,0.400,0.683,-0.045,64.9,-1.42,14147,9002


In [30]:
if_exists = 'replace'
# 데이터베이스 계정과 사용자를 사용하여 Snowflake에 연결 생성

account_identifier = mysql_info['account_identifier']
user = mysql_info['user']
password = mysql_info['password']
conn_string = f"mysql://{user}:{password}@{account_identifier}:3306/baseball"
engine = create_engine(conn_string)

table_name = 'df_player_hitter'

with engine.connect() as con:
    hitter_df.to_sql(
        name = table_name.lower(),
        con = con,
        if_exists = if_exists
    )

In [31]:
data = []

for team in teams:
    # 요청에 필요한 데이터 설정
    url = f'https://statiz.sporki.com/stats/?m=main&m2=pitching&m3=default&so=&ob=&year=2024&sy=2019&ey=2024&te={team}&po=&lt=10100&reg=C3&pe=&ds=&de=&we=&hr=&ha=&ct=&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=1000&ph=&hs=&us=&na=&ls=1&sf1=G&sk1=&sv1=&sf2=G&sk2=&sv2='

    response = requests.post(url, headers=headers)  # 또는 requests.get() 사용

    res = requests.get(url, headers=headers)
    res.raise_for_status()

    html = response.text

    # 응답에서 데이터 추출
    soup = BeautifulSoup(html, "html.parser")

    # print(soup)

    rows = []

    trs = soup.find_all("tr")

    for tr in trs[2:]:
        tds = tr.find_all("td")
        player_id = tds[1].find("a").attrs['href'].split('/')[2].split('&')[1].split('=')[1]
        team_id = tds[2].find("img").attrs['src'].split('/')[5].split('.')[0]
        tds = [td.text for td in tds]
        tds.append(player_id)
        tds.append(team_id)
        rows.append(tds)

    for row in rows:
        temp = []
        for index in range(1, len(row)):
            if index == 2:
                season = row[index][:2]
                position = row[index][3:]
                temp.append(season)
                temp.append(position)
            else:
                temp.append(row[index])
        data.append(temp)

In [32]:
pitcher_df = pd.DataFrame(data, columns=['이름', '연도', '포지션', 'WAR', 'G', 'GS', 'GR', 'GF', 'CG', 'SHO', 'W', 'L', 'S', 'HD', 'IP', 'ER', 'R', 'rRA', 'TBF', 'H', '2B', '3B', 'HR', 'BB', 'HP', 'IB', 'SO', 'ROE', 'BK', 'WP', 'ERA', 'RA9', 'rRA9', 'rRA9pf', 'FIP', 'WHIP', 'DUMMY', 'P_CODE', 'T_CODE'])

pitcher_df = pitcher_df.dropna(axis=0)

pitcher_df.head()

,이름,연도,포지션,WAR,G,GS,GR,GF,CG,SHO,W,L,S,HD,IP,ER,R,rRA,TBF,H,2B,3B,HR,BB,HP,IB,SO,ROE,BK,WP,ERA,RA9,rRA9,rRA9pf,FIP,WHIP,DUMMY,P_CODE,T_CODE
0,안우진,22,P,8.45,30,30,0,1,1,0,15,8,0,0,196.0,46,51,50.65,766,131,36,4,4,55,4,1,224,6,1,7,2.11,2.34,2.33,2.36,2.09,0.95,8.45,13091,10001
1,후라도,23,P,7.12,30,30,0,0,0,0,11,8,0,0,183.2,54,64,62.95,763,164,19,0,7,41,9,0,147,15,0,5,2.65,3.14,3.08,3.13,3.15,1.12,7.12,15531,10001
2,안우진,23,P,6.86,24,24,0,0,0,0,9,7,0,0,150.2,40,44,41.25,608,121,25,1,5,38,1,1,164,5,0,8,2.39,2.63,2.46,2.50,2.33,1.06,6.86,13091,10001
3,요키시,22,P,6.25,30,30,0,0,0,0,10,8,0,0,185.1,53,61,59.80,746,169,31,3,8,33,5,0,154,13,0,2,2.57,2.96,2.90,3.06,2.86,1.09,6.25,13940,10001
4,요키시,20,P,6.08,27,27,0,0,0,0,12,7,0,0,159.2,38,53,50.45,640,144,26,3,6,25,6,0,115,15,1,3,2.14,2.99,2.84,3.00,3.24,1.06,6.08,13940,10001


In [33]:
table_name = 'df_player_pitcher'

with engine.connect() as con:
    pitcher_df.to_sql(
        name = table_name.lower(),
        con = con,
        if_exists = if_exists
    )

In [25]:
data = []

for team in teams:
    # 요청에 필요한 데이터 설정
    url = f'https://statiz.sporki.com/stats/?m=main&m2=fielding&m3=default&so=&ob=&year=2024&sy=2019&ey=2024&te={team}&po=&lt=10100&reg=A&pe=&ds=&de=&we=&hr=&ha=&ct=&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=1000&ph=&hs=&us=&na=&ls=1&sf1=G&sk1=&sv1=&sf2=G&sk2=&sv2='

    response = requests.post(url, headers=headers)  # 또는 requests.get() 사용

    res = requests.get(url, headers=headers)
    res.raise_for_status()

    html = response.text

    # 응답에서 데이터 추출
    soup = BeautifulSoup(html, "html.parser")

    # print(soup)

    rows = []

    trs = soup.find_all("tr")

    for tr in trs[2:]:
        tds = tr.find_all("td")
        player_id = tds[1].find("a").attrs['href'].split('/')[2].split('&')[1].split('=')[1]
        team_id = tds[2].find("img").attrs['src'].split('/')[5].split('.')[0]
        tds = [td.text for td in tds]
        tds.append(player_id)
        tds.append(team_id)
        rows.append(tds)

    for row in rows:
        temp = []
        for index in range(1, len(row)):
            if index == 2:
                season = row[index][:2]
                position = row[index][3:]
                temp.append(season)
                temp.append(position)
            else:
                temp.append(row[index])
        data.append(temp)

In [26]:
defense_df = pd.DataFrame(data, columns=['이름', '연도', '포지션', 'WAAwithPOS', 'G', 'GS', 'IP', 'TC', 'PO', 'Ass', 'E', 'F%', 'RF9', 'Range RAA', 'Arm RAA', 'Err RAA', 'DP RAA', 'Bt RAA', 'CS RAA', 'Blk RAA', 'Frm RAA', 'Total RAA', '/144', 'POSAdj', 'RAAwithPOS', 'WAAwoPOS', 'DUMMY', 'P_CODE', 'T_CODE'])

defense_df = defense_df.dropna(axis=0)

defense_df.head()

,이름,연도,포지션,WAAwithPOS,G,GS,IP,TC,PO,Ass,E,F%,RF9,Range RAA,Arm RAA,Err RAA,DP RAA,Bt RAA,CS RAA,Blk RAA,Frm RAA,Total RAA,/144,POSAdj,RAAwithPOS,WAAwoPOS,DUMMY,P_CODE,T_CODE
0,김혜성,22,2B,2.429,127,123,1080.0,674,292,371,11,98.4,5.53,19.75,,0.29,-0.68,,,,,19.36,21.61,5.52,24.88,1.890,2.429,12905,10001
1,이지영,22,C,2.193,137,102,994.2,906,818,77,11,98.8,8.10,-0.11,,-1.32,,0.21,0.01,2.44,10.71,11.92,12.53,10.54,22.46,1.164,2.193,10312,10001
2,이정후,23,CF,1.282,81,80,698.0,210,204,5,1,99.5,2.69,11.46,1.34,0.45,,,,,,13.24,22.18,0.00,13.24,1.282,1.282,12906,10001
3,이정후,20,RF,1.174,107,93,809.2,191,181,8,2,99.0,2.10,16.85,0.52,0.08,,,,,,17.45,17.95,-4.14,13.31,1.539,1.174,12906,10001
4,박동원,19,C,1.079,89,64,608.0,528,487,40,1,99.8,7.80,0.24,,1.34,,0.01,1.84,0.66,0.35,4.44,5.71,6.58,11.02,0.435,1.079,10387,10001


In [27]:
table_name = 'df_player_defense'

with engine.connect() as con:
    defense_df.to_sql(
        name = table_name.lower(),
        con = con,
        if_exists = if_exists
    )